<a href="https://colab.research.google.com/github/fralfaro/MAT281_2024/blob/main/docs/lectures/data_manipulation/pd_01b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pandas II

## Groupby


**Groupby** es un concepto bastante simple. Podemos crear una agrupación de categorías y aplicar una función a las categorías. 

El proceso de groupby se puede resumiren los siguientes pasos:

* **División**: es un proceso en el que dividimos los datos en grupos aplicando algunas condiciones en los conjuntos de datos.
* **Aplicación**: es un proceso en el que aplicamos una función a cada grupo de forma independiente
* **Combinación**: es un proceso en el que combinamos diferentes conjuntos de datos después de aplicar groupby y resultados en una estructura de datos

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/groupby.jpg" width = "600" align="center"/>




Después de dividir los datos en un grupo, aplicamos una función a cada grupo para realizar algunas operaciones que son:

* **Agregación**: es un proceso en el que calculamos una estadística resumida (o estadística) sobre cada grupo. Por ejemplo, Calcular sumas de grupo o medios
* **Transformación**: es un proceso en el que realizamos algunos cálculos específicos del grupo y devolvemos un índice similar. Por ejemplo, llenar NA dentro de grupos con un valor derivado de cada grupo
* **Filtración**: es un proceso en el cual descartamos algunos grupos, de acuerdo con un cálculo grupal que evalúa Verdadero o Falso. Por ejemplo, Filtrar datos en función de la suma o media grupal

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# cargar datos
path = 'https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/lectures/data_manipulation/data/player_info.csv'
df = pd.read_csv(path, sep="," ).dropna()
df['Decade'] = df['year_start'].apply(lambda x: '2000' if x>=2000 else '1900')
df.head()

###  Agrupar por una columna

In [ ]:
# Agrupar por 'Decade' y calcular la suma de la columna 'Open' en cada grupo
agrupado = df.groupby('position')['weight'].mean()
agrupado

### Agrupar por varias columnas

In [ ]:
# Agrupar por 'Year','Month' y calcular la suma de la columna 'Open' en cada grupo
agrupado = df.groupby(['Decade','position'])['weight'].mean()
agrupado

### Aplicar múltiples funciones 

In [ ]:
# Agrupar por 'Year','Month' y calcular la suma,promedio de la columna 'Open' en cada grupo
agrupado = df.groupby(['Decade','position']).agg({'weight': ['sum', 'mean']})
agrupado

### Groupby Apply

In [ ]:
# Definimos una función que calcula el promedio armónico
def promedio_armónico(datos):
    n = len(datos)
    suma_recíprocos = sum(1 / x for x in datos)
    promedio_armónico = n / suma_recíprocos
    return promedio_armónico

# Aplicamos la función
df.groupby(['Decade', 'position'])['weight'].apply(promedio_armónico)

### Groupby Transform

En pandas, el método `transform()` permite aplicar una función de transformación a cada grupo de un objeto groupby. La función de transformación se aplica a cada grupo y el resultado se asigna de vuelta a las filas correspondientes en el DataFrame original.

In [ ]:
df['mean_weight'] = df.groupby(['Decade','position'])['weight'].transform('mean')
df.head()

## Concat

La función `concat()` realiza todo el trabajo pesado de realizar operaciones de concatenación a lo largo de un eje mientras realiza la lógica de conjunto opcional (unión o intersección) de los índices (si los hay) en los otros ejes. Tenga en cuenta que digo "si hay alguno" porque solo hay un único eje posible de concatenación para Series.

In [ ]:
# cargar datos
path = 'data/player_info.csv'
df = pd.read_csv(path, sep="," ).dropna()
df.head()

In [ ]:
# crear datos
df_concat1 = df.loc[lambda x: x['year_start']<2000]
df_concat1.head()

In [ ]:
# crear datos
df_concat2 = df.loc[lambda x: x['year_start']>=2000]
df_concat2.head()

### Concatenar varias tablas con las mismas columnas

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_01.png" width = "400" align="center"/>

In [ ]:
# concatenar mismas columnas
result = pd.concat([df_concat1,df_concat2])

# mostrar resultados
result

### Concatenar varias tablas distintas columnas

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_02.png" width = "400" align="center"/>

In [ ]:
# cambiar nombre 
df_concat2 = df_concat2.rename(columns = {'birth_date':'birth'})

# concatenar mismas columnas
result = pd.concat([df_concat2,df_concat1])

# mostrar resultados
result

## Merge

La función `merge()` se usa para combinar dos (o más) tablas sobre valores de columnas comunes (keys). 

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_04.png" width = "500" align="center"/>


In [ ]:
# cargar datos
path = 'data/player_info.csv'
df = pd.read_csv(path, sep="," ).dropna()
df.head()

**Por un columna**

In [ ]:
# crear datos
cols_merge1 = ['name', 'year_start', 'year_end', 'position']
df_merge1 = df[cols_merge1]
df_merge1.head()

In [ ]:
# crear datos
cols_merge2 = ['name', 'height', 'weight','birth_date', 'college']
df_merge2 = df[cols_merge2]
df_merge2.head()

In [ ]:
# merge por una columna
result = pd.merge(df_merge1, df_merge2, on='name')
result.head()

**Por Varias columnas**

In [ ]:
# crear datos
cols_merge1 = ['name', 'year_start', 'year_end', 'position']
df_merge1 = df[cols_merge1]
df_merge1.head()

In [ ]:
# crear datos
cols_merge2 = ['name', 'year_start', 'year_end', 'height', 'weight','birth_date', 'college']
df_merge2 = df[cols_merge2]
df_merge2.head()

In [ ]:
# merge varias columnas
result = pd.merge(df_merge1, df_merge2, on=['name', 'year_start', 'year_end'])
result.head()

### Tipos de merge

La opción *how* especificica el tipo de cruce que se realizará.

* **left**: usa las llaves solo de la tabla izquierda
* **right**: usa las llaves solo de la tabla derecha
* **outer**: usa las llaves de la unión de  ambas tablas.
* **inner**: usa las llaves de la intersección de  ambas tablas.

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/joins2.png" width = "500" align="center"/>

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_05.png" width = "600" align="center"/>

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_06.png" width = "600" align="center"/>

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_07.png" width = "600" align="center"/>

<img src="https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/docs/lectures/data_manipulation/data_manipulation/images/merge_08.png" width = "600" align="center"/>

In [ ]:
# tipos de merge
cols = ['name', 'year_start', 'year_end']
merge_left = pd.merge(df_merge1, df_merge2, on=cols, how= 'left')
merge_rigth  = pd.merge(df_merge1, df_merge2, on=cols, how= 'right')
merge_inner  = pd.merge(df_merge1, df_merge2, on=cols, how= 'inner')
merge_outer   = pd.merge(df_merge1, df_merge2, on=cols, how= 'outer')

### Problemas de llaves duplicadas

Cuando se quiere realizar el cruce de dos tablas, pero an ambas tablas existe una columna (key) con el mismo nombre, para diferenciar la información entre la columna de una tabla y otra, pandas devulve el nombre de la columna con un guión bajo x (key_x) y otra con un guión bajo y (key_y)



In [ ]:
df.columns

In [ ]:
# crear datos
cols_merge1 = ['name', 'year_start', 'year_end', 'position' ]
df_merge1 = df[cols_merge1]
df_merge1.head()

In [ ]:
# crear datos
cols_merge2 = ['name', 'year_start', 'year_end', 'height' ]
df_merge2 = df[cols_merge2]
df_merge2.head()

In [ ]:
# merge llaves duplicadas
result = pd.merge(df_merge1, df_merge2, on=['name', 'year_start'])
result.head()

## Tipos de Formatos

<img src="https://www.statology.org/wp-content/uploads/2021/12/wideLong1-1.png" align="center" width = "420">


Dentro del mundo de los dataframe (o datos tabulares) existen dos formas de presentar la naturaleza de los datos: **formato wide** y **formato long**. 


Ejemplo, el siguiente **conjunto de datos** representa estadísticas de rendimiento para cuatro equipos (A, B, C y D) en un cierto contexto deportivo. Cada fila corresponde a un equipo y muestra tres medidas diferentes de rendimiento.

| Team | Points | Assists | Rebounds |
|------|--------|---------|----------|
| A    |   88   |    12   |    22    |
| B    |   91   |    17   |    28    |
| C    |   99   |    24   |    30    |
| D    |   94   |    28   |    31    |

La tabla así presentada se encuentra en **wide format**, es decir, donde los valores se extienden a través de las columnas.

Sería posible representar el mismo contenido anterior en **long format**, es decir, donde los mismos valores se indicaran a través de las filas:

| Team | Variable | Value |
|------|----------|-------|
| A    | Points   | 88    |
| A    | Assists  | 12    |
| A    | Rebounds | 22    |
| B    | Points   | 91    |
| B    | Assists  | 17    |
| B    | Rebounds | 28    |
| C    | Points   | 99    |
| C    | Assists  | 24    |
| C    | Rebounds | 30    |
| D    | Points   | 94    |
| D    | Assists  | 28    |
| D    | Rebounds | 31    |




### Formato long a wide

El pivoteo de una tabla corresponde al paso de una tabla desde el formato **long** al formato **wide**. Típicamente esto se realiza para poder comparar los valores que se obtienen para algún registro en particular, o para utilizar algunas herramientas de visualización básica que requieren dicho formato.

En Pandas se utiliza los comandos `pivot` y `pivot_table`. Formato long a wide

El pivoteo de una tabla corresponde al paso de una tabla desde el formato **long** al formato **wide**. Típicamente esto se realiza para poder comparar los valores que se obtienen para algún registro en particular, o para utilizar algunas herramientas de visualización básica que requieren dicho formato.

En Pandas se utiliza los comandos `pivot` y `pivot_table`. 

In [ ]:
# cargar datos
path = 'data/player_info.csv'
df = pd.read_csv(path, sep="," ).dropna().drop_duplicates()
df['Decade'] = df['year_start'].apply(lambda x: '2000' if x>=2000 else '1900')
df.head()

In [ ]:
# pivot: simple
agrupado = df.groupby(['Decade','position'])['weight'].mean().reset_index()
pivot_df = agrupado.pivot(index='Decade', columns='position', values='weight')
pivot_df.head(10)

In [ ]:
# pivot: multiple
agrupado = df.groupby(['Decade','position','height'])['weight'].mean().fillna(0).astype(int).reset_index()
pivot_df = agrupado.pivot(index=['Decade','height'], columns='position', values='weight').fillna(0)
pivot_df.head(10)

In [ ]:
# pivot_table: simple
pivot_df = df.pivot_table(index='Decade', columns='position', values='weight', aggfunc='mean')
pivot_df.head(10)

In [ ]:
# pivot_table: multiple
pivot_df = df.pivot_table(index=['Decade','height'], columns='position', values='weight', aggfunc='mean').fillna(0)
pivot_df.head(10)

### Formato wide a long


El despivotear una tabla corresponde al paso de una tabla desde el formato **wide** al formato **long**. 

Se reconocen dos situaciones:

1. El valor indicado para la columna es **único**, y sólo se requiere definir correctamente las columnas.
2. El valor indicado por la columna **no es único**, y se requiere una iteración más profunda.

Para despivotear un dataframe en Pandas, utilizaremos el comando `melt`.


In [ ]:
cols_index = ['name','year_start','year_end']
pivot_df = df.pivot_table(index=cols_index, columns='position', values='weight', aggfunc='mean').fillna(0).reset_index()
pivot_df.head()

In [ ]:
# aplicar comando melt
df_melt = pd.melt(
    df, 
    id_vars=cols_index, 
    var_name='Type',
    value_name='Value'
)


df_melt = df_melt.drop('Type',axis=1).rename(columns={'Value':'position'})
df_melt.head()

## Referencias


* [Groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)
* [Merge, join, and concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
* [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)